[View in Colaboratory](https://colab.research.google.com/github/BLUENCE/M5_Idol_lyrics/blob/master/NLP/NLP_sample_Jieun_ver03.ipynb)

In [0]:
import pandas as pd 
import numpy as np
import re
from soynlp.tokenizer import RegexTokenizer
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus

In [0]:
# 참고자료 
# http://nbviewer.jupyter.org/github/KMJJ1/hiphop/blob/master/RNN%20-%20LSTM%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20Hiphop%20%EA%B0%80%EC%82%AC%20%EC%83%9D%EC%84%B1.ipynb
# https://github.com/dyelax/encore.ai

In [0]:
%pwd

'/Users/jieun/projects/M5_Idol_lyrics'

In [0]:
# !pip install soynlp
!pip show soynlp

Name: soynlp
Version: 0.0.46
Summary: Unsupervised Korean Natural Language Processing Toolkits
Home-page: https://github.com/lovit/soynlp
Author: Lovit
Author-email: soy.lovit@gmail.com
License: UNKNOWN
Location: /Users/jieun/anaconda3/lib/python3.6/site-packages
Requires: psutil, numpy
Required-by: 


In [0]:
data= pd.read_csv("song_data_fixed.csv")

In [0]:
song = pd.DataFrame(data)

In [0]:
song.head(3)

,Unnamed: 0,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics
0,0,태지 보이스,서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,NaN,269,서태지\r\r\n작사\r\r\n서태지\r\r\n작곡\r\r\n서태지\r\r\n편곡,갈수 없는 곳이란 우리에겐 없어\r\r\n보이는 길밖에도 세상은 있어\r\r\n언제...
1,1,시대유감(時代遺憾),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,타이틀 곡,"3,256",서태지\r\r\n작사\r\r\n서태지\r\r\n작곡\r\r\n서태지\r\r\n편곡,왜 기다려 왔잖아\r\r\n모든 삶을 포기하는 소리를\r\r\n이 세상이 모두 미쳐...
2,2,수시아(誰是我),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,NaN,488,서태지\r\r\n작사\r\r\n서태지\r\r\n작곡\r\r\n서태지\r\r\n편곡,이 세상 그 누구도 나를 닮을 순 없네\r\r\n날 세상에 알릴 거야\r\r\n나 ...


In [0]:
song.columns

Index(['Unnamed: 0', 'title', 'artist', 'album', 'release_date', 'song_genre',
       'is_title', 'like', 'creator', 'lyrics'],
      dtype='object')

In [0]:
song['artist'].value_counts().head()

동방신기     865
키스       839
슈퍼주니어    518
샤이니      469
쿨        375
Name: artist, dtype: int64

In [0]:
song[song['artist'] == '키스'].head(3) # 키스는 누구인가? => 미국가수, 삭제예정

,Unnamed: 0,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics
5088,5088,Rock And Roll All Nite,키스,Torcida 2018 - Nacional e Internacional,2018.06.01,World,`게임 쉬리` OST (`이젠` Metal Ver.) 와 동영상 수록,1,NaN,You show us everything you've got\r\r\nYou kee...
5089,5089,Rock And Roll All Nite,키스,New Year`s Eve Countdown,2017.12.15,기타,NaN,7,NaN,NaN
5090,5090,Calling Dr. Love,키스,Throwback Tunes: Classic Rock,2017.10.20,Pop,NaN,1,NaN,NaN


In [0]:
# 유니크한 가수의 수 
len(song['artist'].unique())

331

In [0]:
# 곡의 수 
song.shape

(19165, 10)

In [0]:
song['lyrics'].isnull().sum()

2788

In [0]:
song = song['lyrics'].dropna().reset_index() #inplace=True 를 하면 원래 데이터프레임에서 날라간다

In [0]:
song['lyrics'].isnull().sum()

0

In [0]:
song

,index,lyrics
0,0,갈수 없는 곳이란 우리에겐 없어\r\r\n보이는 길밖에도 세상은 있어\r\r\n언제...
1,1,왜 기다려 왔잖아\r\r\n모든 삶을 포기하는 소리를\r\r\n이 세상이 모두 미쳐...
2,2,이 세상 그 누구도 나를 닮을 순 없네\r\r\n날 세상에 알릴 거야\r\r\n나 ...
3,3,옛 생각에 까페 문을열고\r\r\n지난 추억을 기억하려했지\r\r\n부드러운 음악소...
4,4,항상 난 지나치기만 했네\r\r\n이젠 난 미련없이 끝내\r\r\n됐어 찾아냈어 난...
5,5,난 알아요이밤이 흐르고 흐르면 누군가가\r\r\n나를 떠나버려야 한다는 그사실을\r...
6,6,너와 함께 한 시간 속에서-\r\r\n \r\r\n너를 처음 만난 그때를\r\r\n...
7,7,환상속의 그대\r\r\n\r\r\n결코! 시간이 멈추어 줄 순 없다 Yo!\r\r\...
8,8,너에게 모든걸 뺏겨버렸던\r\r\n마음이\r\r\n다시 내게 돌아오는 걸\r\r\n...
9,9,니가 아무리 지금 날 좋아한다고 해도\r\r\n그건 지금뿐일지도 몰라\r\r\n왜냐...


## 텍스트 데이터 전처리

In [0]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [0]:
sample_content = song['lyrics'][100]
type(sample_content)

str

In [0]:
sample_content = preprocessing(sample_content)
sample_content[:1000]

'그대여 내게 아픈 말은 하지마 슬프지 않은 미소지며 잠시 떨려오는 마지막 눈빛으로 헤어짐을 간직하라는 이젠 네가없는 내가 힘들어 나를 포기할 수 있기에 너를 내 꿈속에 스치듯 찾아들면 그때 갈께 너의 곁으로 떠난 그대여 슬퍼 하지마 우리 잃어버린 사랑은 영원할꺼야 다른 세상에 기대고 있어 아직 살아있는 나를 기다려줘 우리 헤어짐은 이대로 간직할께 눈물에 흘리지 않고 그리고 바라보면 언제나 내가 있어 그대 떠난 그대여 슬퍼 하지마 우리 잃어버린 사랑은 영원할꺼야 다른 세상에 기대고있어 아직 살아있는 나를 기다려줘 우리 헤어짐은 이대로 간직할게 눈물에 흘리지 않고 그리워 바라보면 언제나 내가있어 그대'

In [0]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time sentences = song['lyrics'].apply(preprocessing)

CPU times: user 2.55 s, sys: 29.9 ms, total: 2.58 s
Wall time: 2.6 s


In [0]:
sentences.head()

0    갈수 없는 곳이란 우리에겐 없어 보이는 길밖에도 세상은 있어 언제나 식지않는 마음이...
1    왜 기다려 왔잖아 모든 삶을 포기하는 소리를 이 세상이 모두 미쳐버릴 일이 벌어질것...
2    이 세상 그 누구도 나를 닮을 순 없네 날 세상에 알릴 거야 나 역시 그 누구를 따...
3    옛 생각에 까페 문을열고 지난 추억을 기억하려했지 부드러운 음악소리마저 내마음을 아...
4    항상 난 지나치기만 했네 이젠 난 미련없이 끝내 됐어 찾아냈어 난 모두가 또 나를 ...
Name: lyrics, dtype: object

## Bigram 만들기

'말이 되는' 가사 생성에 도움받기 위해 자주 나오는 단어들을 묶어 어구로 만들어야 한다.

n-gram 이라는 기법으로 bigram(두 단어씩 묶기), trigram(세 단어씩 묶기) 등의 방법이 있다.

`bigram = gensim.models.Phrases(tokens, min_count=1, threshold=100)`이런식으로 만들 수 있다.

trigram을 하려면 bigram 으로 나온 것을 한번 더 Phrases에 넣고 돌려주는 방식으로 하면 된다.

// to create the bigrams
bigram_model = Phrases(unigram_sentences)

// apply the trained model to a sentence
 for unigram_sentence in unigram_sentences:                
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])

// get a trigram model out of the bigram
trigram_model = Phrases(bigram_sentences)

In [0]:
# 토크나이징 하기 전 sentences 로 txt파일 생성

%%time
USE_PREMADE_TEXT = False

text_filepath = 'all_lyrics_text.txt'
if not USE_PREMADE_TEXT:
    with open(text_filepath, 'w', encoding='utf-8') as f:
        for lyrics in sentences.values:
            if pd.isnull(lyrics): # null값 있다면 그 다음으로 넘어감
                continue
            f.write(lyrics + '\n')
else:
    assert path.exists(text_filepath)

CPU times: user 79.3 ms, sys: 22.6 ms, total: 102 ms
Wall time: 125 ms


In [0]:
# bigram model 저장 


USE_PREMADE_BIGRAM_MODEL = False

all_bigram_model_filepath = 'all_bigram_model'
all_sentences_normalized_filepath = 'all_lyrics_text.txt'

all_unigram_sentences = LineSentence(all_sentences_normalized_filepath)

if not USE_PREMADE_BIGRAM_MODEL:    
    
    all_bigram_model = Phrases(all_unigram_sentences) #phrase냐 아니냐를 판단해줌
    all_bigram_model.save(all_bigram_model_filepath)
    
else:
    all_bigram_model = Phrases.load(all_bigram_model_filepath)

2018-09-11 15:16:53,728 : INFO : collecting all words and their counts
2018-09-11 15:16:53,731 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-09-11 15:16:58,468 : INFO : PROGRESS: at sentence #10000, processed 2161735 words and 749207 word types
2018-09-11 15:17:01,897 : INFO : collected 1187907 word types from a corpus of 3820065 words (unigram + bigrams) and 16371 sentences
2018-09-11 15:17:01,899 : INFO : using 1187907 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2018-09-11 15:17:01,938 : INFO : saving Phrases object under all_bigram_model, separately None
2018-09-11 15:17:04,442 : INFO : saved all_bigram_model


In [0]:
%%time
USE_PREMADE_BIGRAM_SENTENCES = False

all_bigram_sentences_filepath = 'all_sentences_for_word2vec.txt'

if not USE_PREMADE_BIGRAM_SENTENCES:
    
    with open(all_bigram_sentences_filepath, 'w', encoding='utf-8') as f:
        for unigram_sentence in all_unigram_sentences:
            all_bigram_sentence = all_bigram_model[unigram_sentence]
            f.write(' '.join(all_bigram_sentence) + '\n')
else:
    assert path.exists(all_bigram_sentences_filepath)

/Users/jieun/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 15.2 s, sys: 109 ms, total: 15.3 s
Wall time: 15.4 s


## Word2vec 모델 만들기

Skip-gram 방식으로 word2vec 하였다.

Skip-Gram 방식은 CBOW 방식과 반대로 특정한 단어로부터 문맥이 될 수 있는 단어를 예측한다. 

보통 입력 단어 주변의 kk개 단어를 문맥으로 보고 예측 모형을 만드는데 이 kk 값을 window size 라고 한다.


In [0]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.

import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [0]:
%%time
USE_PREMADE_WORD2VEC = False

all2vec_filepath = 'all_word2vec_model'

if not USE_PREMADE_WORD2VEC:
    
    lyrics_for_word2vec = LineSentence(all_bigram_sentences_filepath)

    all2vec = Word2Vec(lyrics_for_word2vec, size=100, window=5, min_count=1, sg=1)
    # sg=0 cbow 1=Skip-Gram Model
    # 100차원으로 가져옴 / 보통 20~100 정도
    # window = 5 앞 5개, 뒤 5개 단어를 보겠다는 뜻
    # window size 작을수록 문법적인 의미가 너무 중요해짐, 클수록 주제 지향적으로 문맥적인 정보를 많이 담게 됨    
    for _ in range(9):
        all2vec.train(lyrics_for_word2vec,total_examples=16377, epochs=1)
        # 총 곡 가사에 총 16377 문장이 있다는 것 명시
        
    all2vec.save(all2vec_filepath)
else:
    all2vec = Word2Vec.load(all2vec_filepath)
all2vec.init_sims()

2018-09-11 15:20:44,681 : INFO : collecting all words and their counts
2018-09-11 15:20:44,684 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-11 15:20:45,371 : INFO : PROGRESS: at sentence #10000, processed 1718729 words, keeping 148677 word types
2018-09-11 15:20:45,932 : INFO : collected 204703 word types from a corpus of 3015649 raw words and 16371 sentences
2018-09-11 15:20:45,934 : INFO : Loading a fresh vocabulary
2018-09-11 15:20:46,758 : INFO : min_count=1 retains 204703 unique words (100% of original 204703, drops 0)
2018-09-11 15:20:46,760 : INFO : min_count=1 leaves 3015649 word corpus (100% of original 3015649, drops 0)
2018-09-11 15:20:47,369 : INFO : deleting the raw counts dictionary of 204703 items
2018-09-11 15:20:47,374 : INFO : sample=0.001 downsamples 27 most-common words
2018-09-11 15:20:47,376 : INFO : downsampling leaves estimated 2882655 word corpus (95.6% of prior 3015649)
2018-09-11 15:20:47,982 : INFO : estimated required m

2018-09-11 15:21:42,237 : INFO : EPOCH 5 - PROGRESS: at 25.06% examples, 214710 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:43,273 : INFO : EPOCH 5 - PROGRESS: at 34.34% examples, 217878 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:44,275 : INFO : EPOCH 5 - PROGRESS: at 43.02% examples, 219835 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:45,318 : INFO : EPOCH 5 - PROGRESS: at 50.67% examples, 217903 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:46,365 : INFO : EPOCH 5 - PROGRESS: at 58.56% examples, 218980 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:47,374 : INFO : EPOCH 5 - PROGRESS: at 66.25% examples, 219534 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:48,391 : INFO : EPOCH 5 - PROGRESS: at 73.43% examples, 219956 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:49,417 : INFO : EPOCH 5 - PROGRESS: at 80.29% examples, 220195 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:21:50,431 : INFO : EPOCH 5 - PROGRESS: at 87.28% examples, 221395 words/s, in_qsiz

2018-09-11 15:22:30,907 : INFO : training on a 3015649 raw words (2882698 effective words) took 12.5s, 229817 effective words/s
2018-09-11 15:22:30,910 : WARNING : Effective 'alpha' higher than previous training cycles
2018-09-11 15:22:30,912 : INFO : training model with 3 workers on 204703 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2018-09-11 15:22:31,996 : INFO : EPOCH 1 - PROGRESS: at 10.12% examples, 220497 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:22:33,054 : INFO : EPOCH 1 - PROGRESS: at 19.10% examples, 230693 words/s, in_qsize 6, out_qsize 0
2018-09-11 15:22:34,083 : INFO : EPOCH 1 - PROGRESS: at 28.72% examples, 235746 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:22:35,151 : INFO : EPOCH 1 - PROGRESS: at 39.04% examples, 236973 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:22:36,207 : INFO : EPOCH 1 - PROGRESS: at 48.00% examples, 237959 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:22:37,271 : INFO : EPOCH 1 - PROGRESS: at 56.81% ex

2018-09-11 15:23:20,844 : INFO : EPOCH 1 - PROGRESS: at 98.63% examples, 228469 words/s, in_qsize 5, out_qsize 0
2018-09-11 15:23:20,966 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-11 15:23:21,028 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-11 15:23:21,048 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-11 15:23:21,051 : INFO : EPOCH - 1 : training on 3015649 raw words (2882576 effective words) took 12.6s, 228186 effective words/s
2018-09-11 15:23:21,053 : WARNING : EPOCH - 1 : supplied example count (16371) did not equal expected count (16377)
2018-09-11 15:23:21,056 : INFO : training on a 3015649 raw words (2882576 effective words) took 12.6s, 228077 effective words/s
2018-09-11 15:23:21,058 : WARNING : Effective 'alpha' higher than previous training cycles
2018-09-11 15:23:21,060 : INFO : training model with 3 workers on 204703 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5

CPU times: user 8min 48s, sys: 3.99 s, total: 8min 52s
Wall time: 3min 3s


In [0]:
all2vec_filepath = 'all_word2vec_model'
all2vec = Word2Vec.load(all2vec_filepath)

2018-09-11 15:23:57,009 : INFO : loading Word2Vec object from all_word2vec_model
2018-09-11 15:23:57,931 : INFO : loading wv recursively from all_word2vec_model.wv.* with mmap=None
2018-09-11 15:23:57,933 : INFO : loading vectors from all_word2vec_model.wv.vectors.npy with mmap=None
2018-09-11 15:23:57,994 : INFO : setting ignored attribute vectors_norm to None
2018-09-11 15:23:57,996 : INFO : loading vocabulary recursively from all_word2vec_model.vocabulary.* with mmap=None
2018-09-11 15:23:57,998 : INFO : loading trainables recursively from all_word2vec_model.trainables.* with mmap=None
2018-09-11 15:23:58,001 : INFO : loading syn1neg from all_word2vec_model.trainables.syn1neg.npy with mmap=None
2018-09-11 15:23:58,117 : INFO : setting ignored attribute cum_table to None
2018-09-11 15:23:58,119 : INFO : loaded all_word2vec_model


In [0]:
# 에러나면 건너뛰어도 되는 부분임 
# total_examples  
# 총 곡 가사에 총 16377 문장이 있다는 것 명시

model.corpus_count

16377

In [0]:
# 총 20만 4703개의 단어들을 얻었다 

a = pd.DataFrame(all2vec.wv.index2word)
len(a)

204703

In [0]:
len(a)

204703

## 100회 이상 등장한 단어들만 가지고 t-SNE 시각화

t-SNE란 차원축소 기법의 하나로 시각화를 위해 주로 사용된다

t-SNE는 고차원에서 가까운 포인트를 저차원에서도 계속 가깝게 두고, 먼 포인트는 계속 멀게 유지시킨다

데이터간의 유사도를 통해 데이터의 패턴을 찾을 수 있다

단어 중 100회 초과 등장한 단어들을 대상으로, word2vec로 얻어진 각 단어의 numpy 행렬값을 매핑시킨 후 2차원 그래프로 시각화하였다.


In [0]:
%%time
words = []
for i in (range(204703)):
    cnt = all2vec.wv.vocab[a[0][i]] # 횟수 카운트
    if cnt.count > 100:
        words.append(a[0][i]) # 횟수 100회 초과하는 단어만 리스트화
    i += 1

CPU times: user 2.79 s, sys: 27 ms, total: 2.82 s
Wall time: 2.89 s


In [0]:
len (words)  # 100회 이상 등장한 단어는 3140개 

3140

In [0]:
# 100회 이상 등장한 단어들에 대해 vector값 가져오기

X = []
for i in words:
    X.append(all2vec.wv[i])

In [0]:
X2 = pd.DataFrame(X, index = words)

In [0]:
# 단어와 각 vector 값으로 데이터 프레임 생성
X2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
you,-0.705365,0.391794,-0.197246,0.049586,-0.021167,-0.323827,-0.163044,0.389152,-0.418769,-0.273438,...,0.403136,0.466988,-0.180873,0.462283,-0.230707,-0.337114,-0.424592,-0.181719,-0.128530,-0.276971
난,-0.317604,0.064615,-0.357556,-0.482136,-0.121594,-0.302870,0.518841,0.289600,-0.596155,-0.238924,...,0.043901,0.286696,-0.407465,-0.171813,-0.048432,0.198343,0.171555,-0.406401,0.134302,-0.292124
I,-0.568430,0.610572,-0.442661,0.016073,-0.395260,-0.029218,0.391157,0.585260,0.004106,0.166876,...,0.341181,-0.252807,-0.309727,0.016246,-0.074286,-0.281652,-0.142109,-0.356793,0.146877,0.282610
내,-0.454561,-0.136615,-0.467783,-0.500490,0.206679,-0.235675,0.658305,-0.288216,-0.261776,-0.570032,...,-0.266372,0.027186,0.119860,-0.298427,0.333138,-0.066748,0.005722,-0.690182,-0.241884,-0.064313
날,-0.746987,-0.144306,-0.172632,-0.282175,0.231667,-0.297856,0.145883,0.369615,-0.274195,-0.662311,...,-0.274271,0.532227,-0.336424,-0.330417,-0.026896,-0.180967,0.393605,0.058947,-0.198354,-0.079493


In [0]:
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

# t-SNE 모델 모델링 

USE_PREMADE_TSNE = False

tsne_filepath = 'tsne.pkl'

if not USE_PREMADE_TSNE:
    
    tsne = TSNE(random_state=0)
    tsne_points = tsne.fit_transform(X2)
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne_points, f)
else:
    with open(tsne_filepath, 'rb') as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=X2.index, columns=['x_coord', 'y_coord'])
tsne_df['word'] = tsne_df.index

In [0]:
# t-SNE 모델 시각화

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()

# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show time!
show(tsne_plot);

Loading BokehJS ...

각 데이터 포인트에 마우스를 올리면 어떤 단어들이 함께 맵핑되어 있는지 확인 가능하다. 

1. 진한 부분일 수록 비슷한 단어들이 많이 모여 있음을 알 수 있다.


2. 한글, 영어, 일본어, 중국어들은 알아서 끼리끼리 모여서 임베딩되었다. 


3. 전처리를 1도 안한거 치고는 잘 나온다.

## (주의) 100회 이상 단어를 거르지 않고 bigram으로 학습한 전체를 사용해봤더니 상태가 안좋다. 

그냥 토크나이징하고, word2vec 돌리는게 훨씬 잘 나온다. 왜 그런지는 모르겠음 

In [0]:
all2vec.most_similar(positive=['친구'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('꺽다리', 0.6371358036994934),
 ('범생이', 0.6322770714759827),
 ('홀쭉한', 0.6298205256462097),
 ('언변', 0.6060096621513367),
 ('술고래', 0.6045137047767639),
 ('보석이라구', 0.6043168306350708),
 ('느려터진', 0.5892539620399475),
 ('꼬맹이', 0.5878541469573975),
 ('이보게', 0.5690245032310486),
 ('친구냐', 0.5668486952781677)]

In [0]:
all2vec.most_similar(positive=['여자는'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('첨에는', 0.6622425317764282),
 ('목이긴', 0.6306084990501404),
 ('여자답게', 0.6246604323387146),
 ('날려줄게', 0.6172682046890259),
 ('모딜리아니야', 0.6172509789466858),
 ('여잔', 0.6100303530693054),
 ('여자나', 0.60744708776474),
 ('여자라', 0.6012023091316223),
 ('이용하는', 0.6000898480415344),
 ('홈런', 0.5999051332473755)]

In [0]:
all2vec.most_similar(positive=['남자는'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('왠만한', 0.6072508692741394),
 ('차지도', 0.6011484265327454),
 ('입보다', 0.5951322317123413),
 ('타입의', 0.5943436622619629),
 ('짐승이라', 0.5872534513473511),
 ('다짰어', 0.5795607566833496),
 ('건든', 0.578988790512085),
 ('헤프니', 0.5779929757118225),
 ('이상형이야', 0.5757358074188232),
 ('여자맘을', 0.5710512399673462)]

In [0]:
all2vec.most_similar(positive=['여자란'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('다그래', 0.7713031768798828),
 ('좋아하면서도', 0.746355414390564),
 ('말야날이면', 0.7343888282775879),
 ('만나보는그런', 0.7236257195472717),
 ('비밀인데_이제는', 0.7037424445152283),
 ('응원하고', 0.7013207674026489),
 ('접어둬', 0.6976417899131775),
 ('의식한', 0.6959859132766724),
 ('만난거야', 0.6958367824554443),
 ('이기적이라', 0.6911845207214355)]

In [0]:
all2vec.most_similar(positive=['여잔'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('별로거든', 0.6608653664588928),
 ('나보다_착한', 0.6605855226516724),
 ('어쩜_그리', 0.6604582667350769),
 ('안필요해', 0.6561477184295654),
 ('몇명이나', 0.6539269089698792),
 ('뭐냐고', 0.6536110639572144),
 ('청각이', 0.6514410972595215),
 ('예쁜_세상에서', 0.6467814445495605),
 ('가지고_놀기만', 0.6381958723068237),
 ('사귀었던', 0.6366645097732544)]

In [0]:
all2vec.most_similar(positive=['남잔'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('다_믿어서는', 0.6538938879966736),
 ('안된다고_하면서', 0.634789228439331),
 ('세상에서_너보다', 0.6341723203659058),
 ('내_여자니까', 0.625702977180481),
 ('상관없대', 0.6224822998046875),
 ('믿고사는', 0.62077796459198),
 ('남자', 0.6184302568435669),
 ('사랑해줄', 0.6182538866996765),
 ('전부다가', 0.6180988550186157),
 ('좋단', 0.6167598366737366)]

In [0]:
all2vec.most_similar(positive=['girl'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('So_pretty', 0.6447728872299194),
 ('bad_girl', 0.6361664533615112),
 ('boy', 0.6336680054664612),
 ('little', 0.6307015419006348),
 ('Pretty', 0.6153879165649414),
 ('네무레바이이사', 0.6050360798835754),
 ('gir_gir', 0.6003468632698059),
 ('friend_friend', 0.5982957482337952),
 ('나이테시맛테', 0.5948503613471985),
 ('uh_gimme', 0.5921260714530945)]

In [0]:
all2vec.most_similar(positive=['boy'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Mamama', 0.635223388671875),
 ('girl', 0.6336680054664612),
 ('Casacasa', 0.6120827794075012),
 ('my', 0.6018823981285095),
 ('lover', 0.6009724140167236),
 ('살짝만', 0.5958521366119385),
 ('boy_착한', 0.5881478786468506),
 ('luv', 0.5869681239128113),
 ('hurry_up', 0.5855915546417236),
 ('완전_반했나', 0.5850681662559509)]

In [0]:
all2vec.most_similar(positive=['예쁜'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('말만_해요', 0.6039035320281982),
 ('입술은_예쁜', 0.603446900844574),
 ('나만_바라보네요', 0.5815451741218567),
 ('인형을', 0.5645400881767273),
 ('예쁜_여자들은', 0.5633821487426758),
 ('좁아', 0.5599956512451172),
 ('촛불들을', 0.5556084513664246),
 ('생김새와는', 0.554925799369812),
 ('미소', 0.5545122623443604),
 ('살며시_다가왔지', 0.5532582402229309)]

In [0]:
all2vec.most_similar(positive=['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('쭉빠진', 0.7695958018302917),
 ('만나보니', 0.7666535377502441),
 ('섹시함', 0.7590886354446411),
 ('수영복에', 0.7533947229385376),
 ('수영장엔', 0.7325882315635681),
 ('비키니', 0.7306492328643799),
 ('몸매가', 0.7268643975257874),
 ('맵시나는', 0.7098362445831299),
 ('뭣보다', 0.705554187297821),
 ('잘록한', 0.7013905048370361)]

In [0]:
all2vec.most_similar(positive=['오빤'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('애야', 0.7416201233863831),
 ('안들려요', 0.7361965775489807),
 ('이얼싼쓰', 0.7285964488983154),
 ('녹이는', 0.68422532081604),
 ('번지르르해', 0.6826929450035095),
 ('의견', 0.6787101030349731),
 ('넘었어', 0.6768316030502319),
 ('스타일을', 0.6745812296867371),
 ('끌리', 0.6735872626304626),
 ('네게네게네게', 0.6708297729492188)]

In [0]:
model.wv.most_similar('키스')

[('보라빛처럼', 0.8268566727638245),
 ('커피잔', 0.8117149472236633),
 ('그날들이', 0.8078254461288452),
 ('두눈', 0.8053824305534363),
 ('키스도', 0.8042495250701904),
 ('다가왔지', 0.7996252179145813),
 ('바라보아도', 0.7959368824958801),
 ('유머만', 0.7951062321662903),
 ('손가락에', 0.7936034202575684),
 ('행복하기만', 0.7925342917442322)]

In [0]:
model.wv.most_similar('lady')

[('friend', 0.7407693862915039),
 ('boy', 0.7382819652557373),
 ('Pity', 0.7043216228485107),
 ('girl', 0.6931123733520508),
 ('dear', 0.6911106109619141),
 ('Lady', 0.6904165148735046),
 ('boo', 0.6769155859947205),
 ('baby', 0.6719523668289185),
 ('god', 0.6684812307357788),
 ('princess', 0.6532066464424133)]

In [0]:
model.wv.most_similar('')

[('fool', 0.6536678075790405),
 ('freak', 0.6403539776802063),
 ('awinner', 0.6400405168533325),
 ('guy', 0.6358413100242615),
 ('lotta', 0.6308550238609314),
 ('tae', 0.628423273563385),
 ('Man', 0.6247324347496033),
 ('햐쿠닌잇타테', 0.6202173233032227),
 ('saviour', 0.6193612813949585),
 ('doctor', 0.6154305338859558)]

## 최근노래로 갈 수록 가중치를 줘야 할까? 
최근 아이돌의 성향을 보고싶으면,최근것만 보고 해봐야 할 것 같다. 

## 토크나이징

In [0]:
tokenizer = RegexTokenizer()
tokenizer

NameError: name 'RegexTokenizer' is not defined

In [0]:
# 전처리 이후의 샘플 텍스트로 토큰화
tokened_content = tokenizer.tokenize(sample_content)
tokened_content

['그대여',
 '내게',
 '아픈',
 '말은',
 '하지마',
 '슬프지',
 '않은',
 '미소지며',
 '잠시',
 '떨려오는',
 '마지막',
 '눈빛으로',
 '헤어짐을',
 '간직하라는',
 '이젠',
 '네가없는',
 '내가',
 '힘들어',
 '나를',
 '포기할',
 '수',
 '있기에',
 '너를',
 '내',
 '꿈속에',
 '스치듯',
 '찾아들면',
 '그때',
 '갈께',
 '너의',
 '곁으로',
 '떠난',
 '그대여',
 '슬퍼',
 '하지마',
 '우리',
 '잃어버린',
 '사랑은',
 '영원할꺼야',
 '다른',
 '세상에',
 '기대고',
 '있어',
 '아직',
 '살아있는',
 '나를',
 '기다려줘',
 '우리',
 '헤어짐은',
 '이대로',
 '간직할께',
 '눈물에',
 '흘리지',
 '않고',
 '그리고',
 '바라보면',
 '언제나',
 '내가',
 '있어',
 '그대',
 '떠난',
 '그대여',
 '슬퍼',
 '하지마',
 '우리',
 '잃어버린',
 '사랑은',
 '영원할꺼야',
 '다른',
 '세상에',
 '기대고있어',
 '아직',
 '살아있는',
 '나를',
 '기다려줘',
 '우리',
 '헤어짐은',
 '이대로',
 '간직할게',
 '눈물에',
 '흘리지',
 '않고',
 '그리워',
 '바라보면',
 '언제나',
 '내가있어',
 '그대']

In [0]:
%time tokens = sentences.apply(tokenizer.tokenize)

CPU times: user 24 s, sys: 293 ms, total: 24.3 s
Wall time: 24.6 s


In [0]:
tokens[:3]

0    [갈수, 없는, 곳이란, 우리에겐, 없어, 보이는, 길밖에도, 세상은, 있어, 언제...
1    [왜, 기다려, 왔잖아, 모든, 삶을, 포기하는, 소리를, 이, 세상이, 모두, 미...
2    [이, 세상, 그, 누구도, 나를, 닮을, 순, 없네, 날, 세상에, 알릴, 거야,...
Name: lyrics, dtype: object

In [0]:
# 워드투벡한 단어의 사전 수 보기 
len(model.wv.vocab)

162387

In [0]:
# 단어 사전에서 상위 10개만 보기
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:10]

['I', 'you', '내', '난', 'me', 't', '널', '날', '수', 'it']

In [0]:
# 가장 유사한 단어를 추출
model.wv.most_similar('너')

[('완성되지', 0.5744380354881287),
 ('억하심정인지', 0.5725435614585876),
 ('너야', 0.5635101199150085),
 ('물어볼게', 0.5609223246574402),
 ('있기', 0.5590577125549316),
 ('털고도', 0.5531821846961975),
 ('피땀에', 0.551423966884613),
 ('옛정', 0.5352168083190918),
 ('네가', 0.5340349674224854),
 ('하나만', 0.5245047807693481)]

In [0]:
model.wv.most_similar('사랑')

[('사랑은', 0.6728729009628296),
 ('사랑을', 0.6576943397521973),
 ('사랑으로', 0.6437615156173706),
 ('행복을', 0.633502721786499),
 ('사랑아', 0.6203153729438782),
 ('마음', 0.613693356513977),
 ('바래', 0.6091277599334717),
 ('바램', 0.6004109382629395),
 ('없는거라고', 0.5848444700241089),
 ('평생', 0.5808133482933044)]

In [0]:
model.wv.most_similar('여자')

[('남자', 0.84284508228302),
 ('남자를', 0.7608194351196289),
 ('놈이', 0.7475835084915161),
 ('여자는', 0.7387921214103699),
 ('여자야', 0.7235146760940552),
 ('남자가', 0.7207379937171936),
 ('놈', 0.7175110578536987),
 ('여잘', 0.7169695496559143),
 ('남자야', 0.7162264585494995),
 ('여자가', 0.7151209115982056)]

In [0]:
model.wv.most_similar('남자')

[('여자', 0.84284508228302),
 ('여잔', 0.7740013003349304),
 ('남자가', 0.7644602060317993),
 ('여자야', 0.7559769153594971),
 ('놈', 0.7550831437110901),
 ('사람은', 0.7539772391319275),
 ('여잘', 0.753886342048645),
 ('여자를', 0.7496883273124695),
 ('놈이', 0.742708683013916),
 ('남자를', 0.7252805233001709)]

In [0]:
model.wv.most_similar('baby')

[('lady', 0.6719524264335632),
 ('honey', 0.6599240899085999),
 ('babe', 0.6590299606323242),
 ('뿐이기에', 0.6517203450202942),
 ('shiMega', 0.6356972455978394),
 ('girl', 0.6356855630874634),
 ('Baby', 0.6232913136482239),
 ('shiranpuri', 0.6048318147659302),
 ('눈뜨지마', 0.5972138047218323),
 ('아루요니', 0.596859335899353)]

In [0]:
model.wv.most_similar('이별')

[('이별은', 0.7862210869789124),
 ('않잖냐', 0.758197546005249),
 ('슬픈', 0.7512339353561401),
 ('끝내', 0.7437535524368286),
 ('얘기', 0.7389219999313354),
 ('이별을', 0.7288054823875427),
 ('이별이', 0.7193502187728882),
 ('아니였겠지', 0.7178677916526794),
 ('어리석은', 0.7177433371543884),
 ('지나간', 0.712112307548523)]

In [0]:
model.wv.most_similar('sexy')

[('pretty', 0.7721747159957886),
 ('bling', 0.683236837387085),
 ('gorgeous', 0.6781123280525208),
 ('Sexy', 0.6679773926734924),
 ('ome', 0.6646952033042908),
 ('sugar', 0.6584035158157349),
 ('caress', 0.6559441685676575),
 ('Warring', 0.6523290872573853),
 ('ter', 0.6519011855125427),
 ('Paras', 0.6474953889846802)]

In [0]:
model.wv.most_similar('여자친구')

[('예쁘더라', 0.8774560689926147),
 ('홀렸네', 0.8485866189002991),
 ('연락했니', 0.8478046655654907),
 ('얘기만', 0.8463076949119568),
 ('수첩이야', 0.8446948528289795),
 ('번호', 0.8440446853637695),
 ('연락처에', 0.8438520431518555),
 ('그녀완', 0.8435104489326477),
 ('소식', 0.8411042094230652),
 ('멀리한건', 0.8399530649185181)]

In [0]:
model.wv.most_similar('남자친구')

[('행세', 0.8161464333534241),
 ('수이수', 0.8149913549423218),
 ('여자친구', 0.8035971522331238),
 ('소식', 0.7937813997268677),
 ('끊어버리고', 0.7933464050292969),
 ('Isis', 0.7933018207550049),
 ('쿠라야미니이루요데', 0.7877689599990845),
 ('번호', 0.7852747440338135),
 ('올라온', 0.7846376895904541),
 ('피곤해', 0.7839388847351074)]

In [0]:
model.wv.most_similar('친구')

[('얘기', 0.7857072949409485),
 ('좋아요로', 0.747502326965332),
 ('친구가', 0.7441698312759399),
 ('친구들', 0.7215195894241333),
 ('얘길', 0.7154452204704285),
 ('빅뱅이', 0.7145744562149048),
 ('얘기를', 0.7116910219192505),
 ('애기가', 0.6933189630508423),
 ('남자들', 0.691458523273468),
 ('수는없어', 0.6835935711860657)]

## LSTM 공부하고 학습 시켜보기 

코드참고 : https://github.com/KMJJ1/hiphop/blob/master/test_example.ipynb

Instructions to Train:
If you want to use our model to train your own artists, follow these steps:

Pick an artist – it should be someone with a lot of lyrics. (Over 100,000 words).
Collect all of the artist's lyrics from your favorite lyrics website. Save each song as a text file in data/artist_name/. We recommend leaving newlines in as a special token so that the network will learn line and stanza breaks.
Train by navigating to the code directory and running python runner.py -a <artist_name> -m <model_save_name>.
Our models were all trained for 30,000 steps.
Generate new songs by running 
python runner.py -a <artist_name> -l ../save/models/<model_save_name>/<ckpt_file> -t.
Optional: If you would like to specify "prime text" – the initial text that the model will generate from – pass in a string with the -p flag.
Share your trained models with us so we can feature them on our website! Create an issue with a link to a public Dropbox or Google Drive containing your model's .ckpt file.

RNN - LSTM으로 가사 생성¶

일반적인 뉴럴넷에 비해 RNN은 파라미터가 공유되며, backpropagation through time 이라는 특징이 있다
하지만 기존의 RNN은 긴 시퀀스를 잘 처리하지 못하며, 그라디언트가 소멸되는 현상이 발생함
LSTM이라는 RNN의 변형 모델이 등장
LSTM은 '게이트'를 통해 상태값을 계산 시 내부의 상태값을 얼마나 기억하고 사용할지 정한다
그라디언트 흐름을 좋게 만들고 베니싱 현상을 완화시켜 먼 과거에 대한 단어나 정보도 잘 기억할 수 있게 되었다
1) 1000곡의 가사로 학습시킴
Hiphop장르 1000곡의 가사를 RNN - LSTM 방식으로 학습
23만회 학습시킨 모델을 생성
초반 단어 3,4개를 입력하여 차후 나올 단어들을 예측

In [0]:
#!pip3 install tensorflow  

In [0]:
#!pip install --upgrade tensorflow

In [0]:
# -*- coding: utf-8 -*-
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np

from utils import TextLoader

# 학습에 필요한 설정값들을 지정합니다.
data_dir = 'data/tinyshakespeare' # 셰익스피어 희곡 <리처드 3세> 데이터로 학습
#data_dir = 'data/linux' # <Linux 소스코드> 데이터로 학습
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 2
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# vocab_size -> hidden_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime[-1]   # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)